In [ ]:
%%bash

# CI has already allocated a job for the runner.
# Therefore we take all the relevant info from the already initialized env vars.
ACCOUNT_=`sacct --format=account -j $SLURM_JOBID | tail -n1`

echo "ACCOUNT_" ${ACCOUNT_//[[:blank:]]/}
echo "SLURM_JOB_USER" $SLURM_JOB_USER
echo "SLURM_PARTITION" $SLURM_PARTITION

# *****************************************************************************
# Strong Scaling
# *****************************************************************************
# --constraint=clx, i.e., Cascade Lake nodes (Intel codename clx)
# --cpus-per-task=2 -> no multithreading, i.e., one process/task every physical core
# --exclusive, i.e., exclusive use of the node
# --mem=0, i.e., use the whole memory of the node
common_strong_scaling_params="--account=${ACCOUNT_//[[:blank:]]/} --partition=$SLURM_PARTITION --constraint=clx \
                              --cpus-per-task=2 --exclusive --mem=0 --output=%x.out strong_scaling.batch"